In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [78]:
# Scrape website
# html=browser.html

# Find all the car listings on the page
# car_list=car_soup.find_all('div', class_='vehicle-card')

# Testing scraping for the first listing on the site
# car_model=car_soup.find('h2', class_='title')
# car_status=car_soup.find('p', class_='stock-type')
# car_price=car_soup.find('span', class_='primary-price')
# car_dealer=car_soup.find('div', class_='dealer-name')
# dealer_city=car_soup.find('div', class_='miles-from')

# print(car_model, car_price, car_dealer, car_status, dealer_city)


In [79]:
# Testing to make sure text that is stripped is correct
#model=car_model.get_text(strip=True)
#status=car_status.get_text(strip=True)
#price=car_price.get_text(strip=True)
#dealer=car_dealer.get_text(strip=True)
#city=dealer_city.get_text(strip=True)

#print(model, status, price, dealer, city)

In [2]:
# Set up Splinter
browser = Browser('chrome')

# Base URL and initial page
base_url = 'https://www.cars.com/shopping/results/?makes[]=subaru&maximum_distance=150&models[]=subaru-outback&page={}&page_size=100&stock_type=all&zip=61606'
page_number = 1
url = base_url.format(page_number)

# Create lists to store details
models = []
statuses = []
mileages = []
prices = []
dealers = []
cities = []
urls = []  # List to store URLs

# Set to store unique identifiers of scraped cars
scraped_car_ids = set()

# Function to scrape current page
def scrape_page(url):
    browser.visit(url)
    time.sleep(5)  # Adjust as necessary
    
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    if not car_list:
        print("No more car listings found. Exiting.")
        return False

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer_div = car.find('div', class_='dealer-name')
        if car_dealer_div:
            car_dealer = car_dealer_div.find('strong')
            dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'
        else:
            dealer = 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Extract car URL
        car_url = car.find('a', class_='vehicle-card-link')['href'] if car.find('a', class_='vehicle-card-link') else 'N/A'

        # Check if this car has already been scraped
        if car_url in scraped_car_ids:
            continue  # Skip if already scraped
        
        # Add car URL to set of scraped car IDs
        scraped_car_ids.add(car_url)

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)
        urls.append(car_url)  # Append URL to list

    return True

# Counter to track consecutive pages with no new data
no_new_data_count = 0
max_no_new_data_allowed = 3  # Adjust as necessary

# Timeout settings
timeout_seconds = 300  # 5 minutes timeout
start_time = time.time()

# Loop through pages
while True:
    if time.time() - start_time > timeout_seconds:
        print(f"Timeout ({timeout_seconds} seconds) exceeded. Exiting.")
        break
    
    if not scrape_page(url):
        # Increment no_new_data_count if no new data found
        no_new_data_count += 1
        if no_new_data_count >= max_no_new_data_allowed:
            print(f"No new data found for {max_no_new_data_allowed} consecutive pages. Exiting.")
            break
    else:
        # Reset no_new_data_count if new data found
        no_new_data_count = 0
    
    # Increment page number and update URL
    page_number += 1
    url = base_url.format(page_number)
    print(f"Scraping page {page_number}...")

# Print number of cars scraped
print(f"Number of unique cars scraped: {len(scraped_car_ids)}")

Timeout (300 seconds) exceeded. Exiting.
Number of unique cars scraped: 1578


In [2]:
# Printing full page html to look for next-page HTML details
# print(browser.html)

NameError: name 'browser' is not defined

In [3]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities ,
    'URL': urls
    }
car_df= pd.DataFrame(data)

In [4]:
car_df

,Model,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2022 Subaru Outback Wilderness,"17,821 mi.",Used,"$32,977",Victory Toyota of Midtown,"Chicago, IL",/vehicledetail/bd05cd39-732b-4703-b40d-fa2e189...
1,2013 Subaru Outback 2.5i Premium,"87,269 mi.",Used,"$12,950",Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/78f7b565-dde2-468c-b38e-dea74c4...
2,2021 Subaru Outback Touring XT,"40,174 mi.",Used,"$29,000",Ford of Galesburg,"Galesburg, IL (45 mi.)",/vehicledetail/67f52d5f-0b54-4216-9c7e-0475733...
3,2023 Subaru Outback Limited,"11,134 mi.",Used,"$33,995",Schimmer Ford Hyundai,"Peru, IL (52 mi.)",/vehicledetail/345f060d-11a1-44b2-8bfd-4657bba...
4,2023 Subaru Outback Limited,"2,574 mi.",Used,"$33,998",CarMax Indianapolis - Offering Express Pickup ...,N/A,/vehicledetail/c37714e3-dcb5-46f6-96e2-f4c8a08...
...,...,...,...,...,...,...,...
1573,2022 Subaru Outback Premium,"72,139 mi.",Used,"$22,800",McLarty Nissan of Little Rock,"Little Rock, AR (441 mi.)",/vehicledetail/16223818-ef82-4e5b-81d0-e19b025...
1574,2020 Subaru Outback Limited,"65,096 mi.",Used,"$25,998",Honda of Jefferson City,"Jefferson City, MO (203 mi.)",/vehicledetail/bb8bad85-2514-459c-93b7-09a6a66...
1575,2020 Subaru Outback Limited,"65,609 mi.",Used,"$24,886",Woodhouse Chrysler Jeep Dodge RAM Sioux City,"Sioux City, IA (372 mi.)",/vehicledetail/3844fa3e-229a-438d-b1a8-494eac4...
1576,2022 Subaru Outback Wilderness,"71,073 mi.",Used,"$29,000",Tim Short Chrysler Dodge Jeep Ram Hyundai - Fiat,"Pikeville, KY (435 mi.)",/vehicledetail/79b4f7d0-827d-4763-8a37-1e01478...


In [5]:
# Checking to make sure df is formatted correctly
print(car_df.to_string())

                                             Model Mileage (mi)            Status Price (USD)                                                                                  Dealer                             City                                                                          URL
0                   2022 Subaru Outback Wilderness   17,821 mi.              Used     $32,977                                                               Victory Toyota of Midtown                      Chicago, IL    /vehicledetail/bd05cd39-732b-4703-b40d-fa2e18953da5/?attribution_type=isa
1                 2013 Subaru Outback 2.5i Premium   87,269 mi.              Used     $12,950                                                          Sam Leman Toyota - Bloomington         Bloomington, IL (38 mi.)                         /vehicledetail/78f7b565-dde2-468c-b38e-dea74c457b29/
2                   2021 Subaru Outback Touring XT   40,174 mi.              Used     $29,000                               

In [6]:
# Close the browser
browser.quit()

In [7]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                                 Model Mileage (mi) Status Price (USD)  \
0       2022 Subaru Outback Wilderness   17,821 mi.   Used     $32,977   
1     2013 Subaru Outback 2.5i Premium   87,269 mi.   Used     $12,950   
2       2021 Subaru Outback Touring XT   40,174 mi.   Used     $29,000   
3          2023 Subaru Outback Limited   11,134 mi.   Used     $33,995   
4          2023 Subaru Outback Limited    2,574 mi.   Used     $33,998   
...                                ...          ...    ...         ...   
1573       2022 Subaru Outback Premium   72,139 mi.   Used     $22,800   
1574       2020 Subaru Outback Limited   65,096 mi.   Used     $25,998   
1575       2020 Subaru Outback Limited   65,609 mi.   Used     $24,886   
1576    2022 Subaru Outback Wilderness   71,073 mi.   Used     $29,000   
1577       2022 Subaru Outback Premium   37,623 mi.   Used     $25,499   

                                                 Dealer  \
0                             Vi

In [8]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
URL             0
dtype: int64



In [9]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', 'Trim'
car_df[['Year', 'Manufacturer', 'Model', 'Trim']] = car_df['Model'].str.split(' ', 3, expand=True)

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean Mileage column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City', 'URL']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
      Year Manufacturer    Model          Trim  Mileage (mi) Status  \
0     2022       Subaru  Outback    Wilderness       17821.0   Used   
1     2013       Subaru  Outback  2.5i Premium       87269.0   Used   
2     2021       Subaru  Outback    Touring XT       40174.0   Used   
3     2023       Subaru  Outback       Limited       11134.0   Used   
4     2023       Subaru  Outback       Limited        2574.0   Used   
...    ...          ...      ...           ...           ...    ...   
1573  2022       Subaru  Outback       Premium       72139.0   Used   
1574  2020       Subaru  Outback       Limited       65096.0   Used   
1575  2020       Subaru  Outback       Limited       65609.0   Used   
1576  2022       Subaru  Outback    Wilderness       71073.0   Used   
1577  2022       Subaru  Outback       Premium       37623.0   Used   

      Price (USD)                                             Dealer  \
0         32977.0                          Victory Toyota 

C:\Users\awsom\AppData\Local\Temp\ipykernel_20624\2381806475.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [10]:
car_df

,Year,Manufacturer,Model,Trim,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2022,Subaru,Outback,Wilderness,17821.0,Used,32977.0,Victory Toyota of Midtown,"Chicago, IL",/vehicledetail/bd05cd39-732b-4703-b40d-fa2e189...
1,2013,Subaru,Outback,2.5i Premium,87269.0,Used,12950.0,Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/78f7b565-dde2-468c-b38e-dea74c4...
2,2021,Subaru,Outback,Touring XT,40174.0,Used,29000.0,Ford of Galesburg,"Galesburg, IL (45 mi.)",/vehicledetail/67f52d5f-0b54-4216-9c7e-0475733...
3,2023,Subaru,Outback,Limited,11134.0,Used,33995.0,Schimmer Ford Hyundai,"Peru, IL (52 mi.)",/vehicledetail/345f060d-11a1-44b2-8bfd-4657bba...
4,2023,Subaru,Outback,Limited,2574.0,Used,33998.0,CarMax Indianapolis - Offering Express Pickup ...,N/A,/vehicledetail/c37714e3-dcb5-46f6-96e2-f4c8a08...
...,...,...,...,...,...,...,...,...,...,...
1573,2022,Subaru,Outback,Premium,72139.0,Used,22800.0,McLarty Nissan of Little Rock,"Little Rock, AR (441 mi.)",/vehicledetail/16223818-ef82-4e5b-81d0-e19b025...
1574,2020,Subaru,Outback,Limited,65096.0,Used,25998.0,Honda of Jefferson City,"Jefferson City, MO (203 mi.)",/vehicledetail/bb8bad85-2514-459c-93b7-09a6a66...
1575,2020,Subaru,Outback,Limited,65609.0,Used,24886.0,Woodhouse Chrysler Jeep Dodge RAM Sioux City,"Sioux City, IA (372 mi.)",/vehicledetail/3844fa3e-229a-438d-b1a8-494eac4...
1576,2022,Subaru,Outback,Wilderness,71073.0,Used,29000.0,Tim Short Chrysler Dodge Jeep Ram Hyundai - Fiat,"Pikeville, KY (435 mi.)",/vehicledetail/79b4f7d0-827d-4763-8a37-1e01478...


In [12]:
print(car_df.to_string())

    Year Manufacturer    Model             Trim  Mileage (mi)            Status  Price (USD)                                                           Dealer                        City
0   2024       Subaru  Outback       Touring XT           NaN               New      45033.0                                              Uftring Ford Subaru             East Peoria, IL
1   2013       Subaru  Outback     2.5i Premium       87269.0              Used      12950.0                                   Sam Leman Toyota - Bloomington    Bloomington, IL (38 mi.)
2   2022       Subaru  Outback  Onyx Edition XT       46577.0              Used      29340.0                                  Uftring Chrysler Dodge Jeep RAM     North Pekin, IL (6 mi.)
3   2021       Subaru  Outback       Touring XT       40174.0              Used      29000.0                                                Ford of Galesburg      Galesburg, IL (45 mi.)
4   2021       Subaru  Outback          Premium       17404.0         

In [11]:
car_df.to_excel("OutbackTest.xlsx")